In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import numpy as np
import time

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')


selected_classes = list(range(20))


train_filter = np.isin(y_train.flatten(), selected_classes)
test_filter = np.isin(y_test.flatten(), selected_classes)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]


y_train = np.array([selected_classes.index(i[0]) for i in y_train])
y_test = np.array([selected_classes.index(i[0]) for i in y_test])


x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 20)
y_test = to_categorical(y_test, 20)


x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))


Building Model

In [3]:
def build_model(pretrained_model, input_shape=(224, 224, 3), num_classes=20):
    base_model = pretrained_model(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


Model 1: **VGG16**

In [7]:
from tensorflow.keras.applications import VGG16

model_vgg16 = build_model(VGG16)
start_time = time.time()
model_vgg16.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_vgg16.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_vgg16.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("VGG16 Test Loss:", loss)
print("VGG16 Test Accuracy:", acc)


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 77s 240ms/step - accuracy: 0.1557 - loss: 2.8344 - val_accuracy: 0.3220 - val_loss: 2.3238
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 64s 185ms/step - accuracy: 0.3224 - loss: 2.2647 - val_accuracy: 0.3985 - val_loss: 2.0338
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 187ms/step - accuracy: 0.3960 - loss: 2.0249 - val_accuracy: 0.4315 - val_loss: 1.8905
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 151ms/step - accuracy: 0.4287 - loss: 1.8919
Model Size: 56.39241027832031
Inference Time: 0.12346413350105286
VGG16 Test Loss: 1.8904694318771362
VGG16 Test Accuracy: 0.43149998784065247


Model 2: **VGG19**

In [8]:
from tensorflow.keras.applications import VGG19

model_vgg19 = build_model(VGG19)
start_time = time.time()
model_vgg19.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_vgg19.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_vgg19.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("VGG16 Test Loss:", loss)
print("VGG16 Test Accuracy:", acc)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 102s 289ms/step - accuracy: 0.1044 - loss: 2.9360 - val_accuracy: 0.2385 - val_loss: 2.5738
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 124s 266ms/step - accuracy: 0.2415 - loss: 2.5128 - val_accuracy: 0.2880 - val_loss: 2.3181
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 134s 240ms/step - accuracy: 0.2994 - loss: 2.3020 - val_accuracy: 0.3480 - val_loss: 2.1530
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step - accuracy: 0.3625 - loss: 2.1483
Model Size: 76.64729309082031
Inference Time: 0.18350576984882355
VGG16 Test Loss: 2.153027057647705
VGG16 Test Accuracy: 0.3479999899864197


Model 3: **RestNet50**

In [9]:
from tensorflow.keras.applications import ResNet50

model_resnet50 = build_model(ResNet50)
start_time = time.time()
model_resnet50.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_resnet50.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_resnet50.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("RestNet50 Test Loss:", loss)
print("RestNet50 Test Accuracy:", acc)


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 54s 131ms/step - accuracy: 0.0491 - loss: 3.0449 - val_accuracy: 0.0500 - val_loss: 2.9958
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 66s 97ms/step - accuracy: 0.0487 - loss: 2.9959 - val_accuracy: 0.0500 - val_loss: 2.9957
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 97ms/step - accuracy: 0.0518 - loss: 2.9960 - val_accuracy: 0.0500 - val_loss: 2.9957
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.0528 - loss: 2.9957
Model Size: 90.99031066894531
Inference Time: 0.08044157457351685
RestNet50 Test Loss: 2.9957449436187744
RestNet50 Test Accuracy: 0.05000000074505806


Model 4: **RestNet101**

In [10]:
from tensorflow.keras.applications import ResNet101

model_resnet101 = build_model(ResNet101)
start_time = time.time()
model_resnet101.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_resnet101.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_resnet101.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("RestNet101 Test Loss:", loss)
print("ResNet101 Test Accuracy:", acc)


171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 98s 241ms/step - accuracy: 0.0592 - loss: 3.0162 - val_accuracy: 0.0805 - val_loss: 2.9712
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 114s 172ms/step - accuracy: 0.0902 - loss: 2.9609 - val_accuracy: 0.0990 - val_loss: 2.9176
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 173ms/step - accuracy: 0.0986 - loss: 2.9099 - val_accuracy: 0.1080 - val_loss: 2.8885
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 138ms/step - accuracy: 0.1162 - loss: 2.8851
Model Size: 163.7383575439453
Inference Time: 0.16088604533672332
RestNet101 Test Loss: 2.888521909713745
ResNet101 Test Accuracy: 0.1080000028014183


Model 5: **NASNetMobile**

In [4]:
from tensorflow.keras.applications import NASNetMobile

model_nasnetmobile = build_model(NASNetMobile)
start_time = time.time()
model_nasnetmobile.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_nasnetmobile.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_nasnetmobile.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("NASNetMobile Test Loss:", loss)
print("NASNetMobileNet Test Accuracy:", acc)


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 93s 181ms/step - accuracy: 0.6138 - loss: 1.3783 - val_accuracy: 0.7975 - val_loss: 0.6611
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step - accuracy: 0.8078 - loss: 0.5873 - val_accuracy: 0.8125 - val_loss: 0.6355
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.8415 - loss: 0.4855 - val_accuracy: 0.8155 - val_loss: 0.5814
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.7978 - loss: 0.6226
Model Size: 16.813629150390625
Inference Time: 0.0763734130859375
NASNetMobile Test Loss: 0.581369161605835
NASNetMobileNet Test Accuracy: 0.815500020980835


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import numpy as np
import time

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')


selected_classes = list(range(20))


train_filter = np.isin(y_train.flatten(), selected_classes)
test_filter = np.isin(y_test.flatten(), selected_classes)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]


y_train = np.array([selected_classes.index(i[0]) for i in y_train])
y_test = np.array([selected_classes.index(i[0]) for i in y_test])


x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 20)
y_test = to_categorical(y_test, 20)


x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))


In [5]:
def build_model(pretrained_model, input_shape=(224, 224, 3), num_classes=20):
    base_model = pretrained_model(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


Model 6: **InceptionV3**

In [5]:
from tensorflow.keras.applications import InceptionV3

model_inception = build_model(InceptionV3)
start_time = time.time()
model_inception.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_inception.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_inception.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("InceptionV3 Test Loss:", loss)
print("InceptionV3 Test Accuracy:", acc)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 57s 127ms/step - accuracy: 0.5867 - loss: 1.4188 - val_accuracy: 0.7710 - val_loss: 0.7214
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 58s 77ms/step - accuracy: 0.8059 - loss: 0.5845 - val_accuracy: 0.7840 - val_loss: 0.6865
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 80ms/step - accuracy: 0.8536 - loss: 0.4408 - val_accuracy: 0.7930 - val_loss: 0.6849
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.7918 - loss: 0.7082
Model Size: 84.18135070800781
Inference Time: 0.07817060291767121
InceptionV3 Test Loss: 0.6848650574684143
InceptionV3 Test Accuracy: 0.7929999828338623


Model 7: **Xception**

In [6]:
from tensorflow.keras.applications import Xception

model_xception = build_model(Xception)
start_time = time.time()
model_xception.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_xception.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_xception.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("Xception Test Loss:", loss)
print("Xception Test Accuracy:", acc)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 77s 209ms/step - accuracy: 0.6431 - loss: 1.2404 - val_accuracy: 0.8010 - val_loss: 0.6183
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 157ms/step - accuracy: 0.8306 - loss: 0.5205 - val_accuracy: 0.8075 - val_loss: 0.5904
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 157ms/step - accuracy: 0.8777 - loss: 0.3714 - val_accuracy: 0.8065 - val_loss: 0.6342
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 122ms/step - accuracy: 0.8002 - loss: 0.6742
Model Size: 80.59056091308594
Inference Time: 0.10412070488929749
Xception Test Loss: 0.6342160701751709
Xception Test Accuracy: 0.8065000176429749


Model 8: **EfficientNetB0**

In [7]:
from tensorflow.keras.applications import EfficientNetB0

model_effnet = build_model(EfficientNetB0)
start_time = time.time()
model_effnet.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_effnet.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_effnet.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("EfficientNetB0 Test Loss:", loss)
print("EfficientNetB0 Test Accuracy:", acc)



16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 56s 108ms/step - accuracy: 0.0467 - loss: 3.0084 - val_accuracy: 0.0500 - val_loss: 2.9958
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 51s 46ms/step - accuracy: 0.0453 - loss: 2.9960 - val_accuracy: 0.0500 - val_loss: 2.9957
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step - accuracy: 0.0449 - loss: 2.9959 - val_accuracy: 0.0500 - val_loss: 2.9957
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.0580 - loss: 2.9956
Model Size: 16.08321762084961
Inference Time: 0.06376260769367217
EfficientNetB0 Test Loss: 2.995741128921509
EfficientNetB0 Test Accuracy: 0.05000000074505806


Model 9: **DenseNet121**

In [6]:
from tensorflow.keras.applications import DenseNet121

model_densenet = build_model(DenseNet121)
start_time = time.time()
model_densenet.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
end_time = time.time()
loss, acc = model_densenet.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_densenet.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("DenseNet121 Test Loss:", loss)
print("DenseNet121 Test Accuracy:", acc)



Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 88s 189ms/step - accuracy: 0.5130 - loss: 1.6319 - val_accuracy: 0.7515 - val_loss: 0.7519
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 86ms/step - accuracy: 0.7932 - loss: 0.6364 - val_accuracy: 0.7665 - val_loss: 0.6927
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 88ms/step - accuracy: 0.8248 - loss: 0.5305 - val_accuracy: 0.8075 - val_loss: 0.6157
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.7918 - loss: 0.6602
Model Size: 27.356277465820312
Inference Time: 0.08376858985424042
DenseNet121 Test Loss: 0.6157269477844238
DenseNet121 Test Accuracy: 0.8075000047683716


Model 10: **MobileNetV2**

In [7]:
from tensorflow.keras.applications import MobileNetV2

model_mobilenetv2= build_model(MobileNetV2)
start_time = time.time()
model_mobilenetv2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)
end_time = time.time()
loss, acc = model_mobilenetv2.evaluate(x_test, y_test)
inference_time = (end_time - start_time) / len(x_test)
model_size = model_mobilenetv2.count_params() * 4 / (1024 ** 2)
print("Model Size:", model_size)
print("Inference Time:", inference_time)
print("MobileNetV2 Test Loss:", loss)
print("MobileNetV2 Test Accuracy:", acc)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 66ms/step - accuracy: 0.5373 - loss: 1.5703 - val_accuracy: 0.7535 - val_loss: 0.7636
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.7789 - loss: 0.6723 - val_accuracy: 0.7585 - val_loss: 0.7642
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.8225 - loss: 0.5347 - val_accuracy: 0.7605 - val_loss: 0.7148
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8557 - loss: 0.4361 - val_accuracy: 0.7775 - val_loss: 0.6678
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8832 - loss: 0.3652 - val_accuracy: 0.7840 - val_loss: 0.6790
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7683 - loss: 0.7399
Model Size: 9.248855590820312
Inference Time: 0.047689895629882816
MobileNetV2 Test Loss: 0.6789664626121521
MobileNetV2 Test Accuracy: 0.7839999794960022
